In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from Liveness_Model import LivenessNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
import tensorflow as tf

In [2]:
# Checking if GPU available
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
dataset = 'live_dataset' # Training Data
model_path = 'liveness.model' # Output Model
plot = 'plot.png' # Output Performance Plot
le1 = 'le.pickle' # Model Network Strucutre

In [4]:
# Initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 8
EPOCHS = 50

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset))
data = []
labels = []

[INFO] loading images...


In [5]:
# loop over all image paths
for imagePath in imagePaths:
    
    # extract the class label from the filename, load the image and
    # resize it to be a fixed 32x32 pixels, ignoring aspect ratio
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32))
    
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

    # convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

In [6]:
# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.25, random_state=78)


# Construct the training image generator for data augmentation
# Generate batches of tensor image data with real-time data augmentation.

# rotation_range	Int. Degree range for random rotations.

# zoom_range: Float or [lower, upper]. Range for random zoom. 
# If a float, [lower, upper] = [1-zoom_range, 1+zoom_range].

# width_shift_range	Float, 1-D array-like or int
# float: fraction of total width, if < 1, or pixels if >= 1.

# height_shift_range	Float, 1-D array-like or int
# float: fraction of total height, if < 1, or pixels if >= 1.

# shear_range	Float. Shear Intensity (Shear angle in 
# counter-clockwise direction in degrees)

# fill_mode	One of {"constant", "nearest", "reflect" or "wrap"}. Default is 'nearest'. Points outside the boundaries of the input are filled according to the given mode:
# 'constant': kkkkkkkk|abcd|kkkkkkkk (cval=k)
# 'nearest': aaaaaaaa|abcd|dddddddd
# 'reflect': abcddcba|abcd|dcbaabcd
# 'wrap': abcdabcd|abcd|abcdabcd

# horizontal_flip: Boolean. Randomly flip inputs horizontally.
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
    width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
    horizontal_flip=True, fill_mode="nearest")

# initialize the optimizer and model
print("[INFO] compiling model...")
# Adam Optimizer
# lr: learning rate
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model = LivenessNet.build(width=32, height=32, depth=3,
    classes=len(le.classes_))

model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
model.summary()

[INFO] compiling model...
[INFO] training network for 50 epochs...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 16)        64        
_______________________________________________________

In [7]:
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1), target_names=le.classes_))

# save the network to disk
print("[INFO] serializing network to '{}'...".format(model))
model.save(model_path, save_format="h5")

# save the label encoder to disk
f = open(le1, "wb")
f.write(pickle.dumps(le))
f.close()


Epoch 1/50
9/9 [==============================] - 1s 104ms/step - loss: 1.0793 - accuracy: 0.5857 - val_loss: 0.6787 - val_accuracy: 0.6667
Epoch 2/50
9/9 [==============================] - 0s 9ms/step - loss: 0.9016 - accuracy: 0.6857 - val_loss: 0.6578 - val_accuracy: 0.7037
Epoch 3/50
9/9 [==============================] - 0s 8ms/step - loss: 0.8657 - accuracy: 0.6571 - val_loss: 0.6524 - val_accuracy: 0.5556
Epoch 4/50
9/9 [==============================] - 0s 9ms/step - loss: 0.6249 - accuracy: 0.7714 - val_loss: 0.6857 - val_accuracy: 0.5556
Epoch 5/50
9/9 [==============================] - 0s 9ms/step - loss: 0.5484 - accuracy: 0.7571 - val_loss: 0.7471 - val_accuracy: 0.5556
Epoch 6/50
9/9 [==============================] - 0s 9ms/step - loss: 0.3158 - accuracy: 0.8143 - val_loss: 0.8514 - val_accuracy: 0.5556
Epoch 7/50
9/9 [==============================] - 0s 9ms/step - loss: 0.3730 - accuracy: 0.8286 - val_loss: 0.9682 - val_accuracy: 0.5556
Epoch 8/50
9/9 [================

In [9]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot)